In [5]:
import Ffmpeg

ModuleNotFoundError: No module named 'Ffmpeg'

In [6]:
import parcels

INFO: Compiled ParcelsRandom ==> /tmp/parcels-2919/libparcels_random_1b231af1-7a57-4d17-aced-3b510a72ffc0.so


In [2]:
%%bash
conda info --envs

# conda environments:
#
base                     /home/rbeutel/anaconda3
OceanParcels          *  /home/rbeutel/anaconda3/envs/OceanParcels
cookiecutter             /home/rbeutel/anaconda3/envs/cookiecutter
eosc510                  /home/rbeutel/anaconda3/envs/eosc510
eosc573                  /home/rbeutel/anaconda3/envs/eosc573

